In [5]:
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

# Initialize the language model
llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo-0125",
)

# Initialize the SQL database connection
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")

# Initialize the SQL database toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Create the SQL agent with the required arguments
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Define the final SQL query without a limit
final_query = """
SELECT m.title, d.name AS director, m.gross_revenue 
FROM movies m 
JOIN directors d ON m.director_id = d.id 
ORDER BY m.vote_count DESC, m.budget ASC;
"""

# Invoke the agent with the final query
response = agent.invoke(final_query)

# Print the output
print(response["output"])



> Entering new SQL Agent Executor chain...
We need to check if the tables "movies" and "directors" exist in the database and also verify the fields in those tables before running the query.
Action: sql_db_list_tables
Action Input: directors, moviesWe have the tables "directors" and "movies" in the database.
Action: sql_db_schema
Action Input: directors, movies
CREATE TABLE directors (
	name TEXT, 
	id INTEGER, 
	gender INTEGER, 
	uid INTEGER, 
	department TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from directors table:
name	id	gender	uid	department
James Cameron	4762	2	2710	Directing
Gore Verbinski	4763	2	1704	Directing
Sam Mendes	4764	2	39	Directing
*/


CREATE TABLE movies (
	id INTEGER, 
	original_title VARCHAR, 
	budget INTEGER, 
	popularity INTEGER, 
	release_date TEXT, 
	revenue INTEGER, 
	title TEXT, 
	vote_average REAL, 
	vote_count INTEGER, 
	overview TEXT, 
	tagline TEXT, 
	uid INTEGER, 
	director_id INTEGER DEFAULT 0 NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from movies table:


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 76903 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}